# Corruption Matrix Benchmark

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Fit SPAM model

### Generate True Model, Full POVM and Inital

In [3]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm.povm)

    #add noise
    if shots != 0:
        targets_spam = add_shot_noise(targets_spam, shots=shots)
        
    return inputs_spam, targets_spam


def spam_pipeline(n, 
                  c1, 
                  c2,
                  shots,
                  use_corr = False,
                  num_iter = 2000):
    spam_target = generate_spam_benchmark(n=n, c1=c1, c2=c2)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    
    if use_corr:
        povm = CorruptionMatrix(d)
    else:
        povm = POVM(d)
        
    spam_model = SPAM(init = InitialState(d),
                      povm = povm,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))

    spam_model.pretrain(num_iter = 500, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter,
                     verbose = False,
                    )
    
    return spam_target, spam_model

## Three Qubit

In [21]:
n = 3
d = 2**n
c1 = 0.997
c2 = 0.89


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=False,
                                                num_iter = 2000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_3qubit_full_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

In [22]:
n = 3
d = 2**n
c1 = 0.997
c2 = 0.89


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=True,
                                                num_iter = 2000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_3qubit_corr_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

## Four Qubit

In [27]:
n = 4
d = 2**n
c1 = 0.96
c2 = 0.88


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=False,
                                                num_iter = 1000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_4qubit_full_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
n = 4
d = 2**n
c1 = 0.96
c2 = 0.88


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=True,
                                                num_iter = 1000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_4qubit_corr_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

c:\Users\krist\anaconda3\envs\env_qiskit\lib\site-packages\tensorflow\python\framework\ops.py:1064: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self._numpy())


FileNotFoundError: [Errno 2] No such file or directory: '../../data\\spam_fidelity_4qubit_corr_shotnoise.data'